<font size="6">Let's predict the new world champion..!</font>

'

In [39]:
import pandas as pd
import pickle 
from scipy.stats import poisson


<font size="3">1.Import the tables</font> 

In [40]:
tables=pickle.load(open('../Web_Scraping /Data/dict_table','rb'))
history=pd.read_csv('../Web_Scraping /Data/Clean_Fifa_worldcup_missing_data.csv')
fixture=pd.read_csv('../Web_Scraping /Data/Clean_Fixture_2022.csv')
history.rename({'HomeAway':'AwayTeam'},axis=1,inplace=True)

<font size="6">2.Calculate Team Strength..</font>

In [41]:
df_home=history[['HomeTeam','HomeGoals','AwayGoals']]
df_away=history[['AwayTeam','AwayGoals','HomeGoals']]
df_home=df_home.rename(columns={'HomeTeam':'Team','HomeGoals':'GoalsScored','AwayGoals':'GoalsConceded'})
df_away=df_away.rename(columns={'AwayTeam':'Team','HomeGoals':'GoalsConceded','AwayGoals':'GoalsScored'})

'

<font size="3">Concatenate df_home and df_away make a groupby by 'Team' and calculate the average..</font> 

In [42]:
df_team_strength=pd.concat([df_home,df_away],ignore_index=True).groupby('Team').mean()

<font size="3">This model was chosen because it fits the Poisson distribution</font> 

<img src="../source/Condiciones.png">

<font size="3">So.. how works?</font> 

<img src="../source/Distribucion de poisson.png">

<font size="3">Having understood the formula. let's apply it to our example!!</font> 

In [43]:
def predict_point(home,away):
    if home in df_team_strength.index and away in df_team_strength.index:
        #goals scored* goals conceded
        lamb_home=df_team_strength.at[home,'GoalsScored']*df_team_strength.at[away,'GoalsConceded']
        lamb_away=df_team_strength.at[away,'GoalsScored']*df_team_strength.at[home,'GoalsConceded']
        prob_home,prob_away,prob_draw=0,0,0
        for x in range(0,11):#Numbers of Goals of Home Team
            for y in range(0,11):#Numbers of Goals of Away Team
                p=poisson.pmf(x,lamb_home)*poisson.pmf(y,lamb_away)
                if x==y:
                    prob_draw+=p
                elif x>y:
                    prob_home+=p
                else:
                    prob_away+=p
        points_home=3*prob_home+prob_draw
        points_away=3*prob_away+prob_draw
        return (points_home,points_away)
    else:
        return(0,0)

<font size="3">Let's test the function! </font> 

In [44]:
predict_point('Bolivia','Argentina')

(0.010097814248235259, 2.894594744185821)

<font size="6">3.Predicting the world cup 2022</font>

<font size="3">3.1 Group stage</font> 

In [45]:
fixture_48=fixture[:48].copy()
fixture_knockout=fixture[48:56].copy()
fixture_quarter=fixture[56:60].copy()
fixture_semi=fixture[60:62].copy()
fixture_final=fixture[62:].copy()

In [46]:
for group in tables:
    teams_in_group = tables[group]['Team'].values
    df_fixture_group_6 = fixture_48[fixture_48['home'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_point(home, away)
        tables[group].loc[tables[group]['Team'] == home, 'Pts'] += points_home
        tables[group].loc[tables[group]['Team'] == away, 'Pts'] += points_away

    tables[group] = tables[group].sort_values('Pts', ascending=False).reset_index()
    tables[group] = tables[group][['Team', 'Pts']]
    tables[group] = tables[group].round(0)

In [51]:
tables['Group C']

,Team,Pts
0,Argentina,7.0
1,Poland,6.0
2,Mexico,4.0
3,Saudi Arabia,1.0
